In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt 
from numpy import asarray
from skimage import transform

img = "../data/raw_data/example_segmentation/ISIC_0001769_segmentation.png"
img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)

In [ ]:
def find_center(img):
    img_bin = (img>128).astype(np.uint8)
    contours, _ = cv2.findContours(img_bin, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)

    M = cv2.moments(contours[0])
    center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    return center

In [ ]:
# Found by Erling Amundsen
def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (find_center(image))
    print(find_center(image))

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

rot_img = rotate_bound(img,45)
plt.imshow(rot_img)
find_center(rot_img)




In [ ]:
plt.imshow(cv2.flip(rot_img,1))

In [ ]:
right_side = img[:,find_center(img)[0]:]
left_side = img[:,:find_center(img)[0]]
plt.imshow(right_side)

In [ ]:
def flip_image(image):
    right_side = image[:,find_center(image)[0]:]
    left_side = cv2.flip(image[:,:find_center(image)[0]],1)
    if right_side.shape[1]>left_side.shape[1]:
        right_side = right_side[:,:left_side.shape[1]]
    else:
        left_side = left_side[:,:right_side.shape[1]]

    up_side = image[:find_center(image)[1],:]
    down_side = cv2.flip(image[find_center(image)[1]:,:],0)
    if up_side.shape[0]>down_side.shape[0]:
        diff = int(np.abs(up_side.shape[0]-down_side.shape[0]))
        up_side = up_side[diff:,:]
    else:
        diff = int(np.abs(up_side.shape[0]-down_side.shape[0]))
        down_side = down_side[diff:,:]
    plt.imshow(down_side)
    print(right_side.shape,left_side.shape,up_side.shape,down_side.shape)

    print(np.sum(right_side),np.sum(left_side))

    dif_x,dif_y = int(np.sum(right_side-left_side)),np.abs(int(np.sum(up_side-down_side)))
    print(dif_x)
    plt.imshow(up_side+down_side)
    return (dif_x+dif_y)//2
(flip_image(img)+flip_image(rot_img))//2







In [ ]:
img = "../data/raw_data/example_segmentation/ISIC_0001769_segmentation.png"
def symmetry(img):
    img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    img = rotate_bound(img, 0)
    rot_img = rotate_bound(img, 45)
    return (flip_image(img)+flip_image(rot_img))//2

symmetry(img)
